In [35]:
import pvlib
import pandas as pd

def get_weather_data(location):
    out = pvlib.iotools.get_pvgis_tmy(location.latitude, location.longitude, map_variables=True)
    weather = out[0]
    weather.index = weather.index.map(lambda x: x.replace(year=2016))
    return weather


def get_solar_position_at(location, weather_timesteps):
    times = weather_timesteps - pd.Timedelta('30min')
    solar_position = location.get_solarposition(times)
    solar_position.index += pd.Timedelta('30min')
    return solar_position;

def get_string_power(string, weather, solar_position):
    parameters = pvlib.temperature.TEMPERATURE_MODEL_PARAMETERS['sapm']['open_rack_glass_polymer']
    string_powers_df = None
    for block in string.blocks:
        irradiance = pvlib.irradiance.get_total_irradiance(surface_tilt=block.tilt,
                                                       surface_azimuth=block.azimuth,
                                                       dni=weather['dni'],
                                                       ghi=weather['ghi'],
                                                       dhi=weather['dhi'],
                                                       solar_zenith=solar_position['apparent_zenith'],
                                                       solar_azimuth=solar_position['azimuth'])
        cell_temperature = pvlib.temperature.sapm_cell(irradiance["poa_global"],
                                                    weather['temp_air'],
                                                    weather['wind_speed'],
                                                    **parameters)
        block_nameplate = block.nameplate_of_one * block.num_panels
        block_gamma_pdc = block.gamma_pdc_of_one * block.num_panels
        block_power = pvlib.pvsystem.pvwatts_dc(irradiance["poa_global"], cell_temperature, block_nameplate, block_gamma_pdc)
        if(string_powers_df is None):
            string_powers_df = block_power
        else:
            string_powers_df = string_powers_df.add(block_power)
    return string_powers_df

def get_system_power(location, strings):
    location = pvlib.location.Location(latitude=location.lat,
                                longitude=location.lon,
                                altitude=location.alt)
    
    weather = get_weather_data(location)

    solar_position = get_solar_position_at(location, weather.index)
    
    system_power_df = None;
    for string in strings:
        string_power = get_string_power(string, weather, solar_position)
        if(system_power_df is None):
            system_power_df = string_power
        else:
            system_power_df = system_power_df.add(string_power)
    return system_power_df

In [62]:
def run(location, strings):
    system_power_df = get_system_power(location, strings)    

    # def get_month(data, month):
    #     return data[data.index.month==month]
    monthly_sum = system_power_df.resample('M').sum()
    monthly_avg_by_time = system_power_df.groupby([system_power_df.index.month, system_power_df.index.time]).mean()

    # Iterate over the groups
    monthly_avgs = {}
    for (month, time), value in monthly_avg_by_time.items():
        if month not in monthly_avgs:
            monthly_avgs[month] = []
        
        # Append the average value to the corresponding month list
        monthly_avgs[month].append(value)
    result_dict = {}
    result_dict["monthly_avgs"] = monthly_avgs
    result_dict["sums_acrross_months"] = monthly_sum.values.tolist()
    return result_dict

In [63]:
from models import LocationModel, PanelBlock, String
location = LocationModel(lat=49.224156,lon=16.5597962,alt=239.25)
strings = [
    String(blocks=[
    PanelBlock(tilt=-2.0850642332041385e-12,num_panels=12, azimuth= 0.9822767414364175, nameplate_of_one= 0.4, gamma_pdc_of_one=  0.001),
    PanelBlock(tilt=-2.0850642332041385e-12, num_panels=1, azimuth=0.9822767414364175, nameplate_of_one= 0.4, gamma_pdc_of_one= 0.001),
    PanelBlock(tilt=-2.0850642332041385e-12,num_panels=3,azimuth= 0.9822767414364175, nameplate_of_one= 0.4, gamma_pdc_of_one= 0.001)
])]
   

In [64]:
run(location, strings)

{'monthly_avgs': {1: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.053701844989045865, 0.3635036652453755, 0.666060962530182, 0.7992231402134995, 0.8522076905613841, 0.7318497387468278, 0.6609871458543845, 0.37951716231918564, 0.004917046524599206, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], 2: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.013645871189346284, 0.2951942286716836, 0.6490290065641929, 0.7872508409343323, 0.9424003558601068, 0.9794867862039264, 0.9151097398762081, 0.8416199761466007, 0.6426685125715086, 0.3882742860724059, 0.00851171583418764, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], 3: [0.0, 0.0, 0.0, 0.0, 0.0, 0.018552094120648496, 0.32044678341577953, 1.001533404313787, 1.7428788309675591, 2.2345758164988014, 2.6085326597428544, 2.6680151402762515, 2.6137557854787024, 2.262404245164015, 1.864541899087937, 1.2482474423851837, 0.503192438491827, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], 4: [0.0, 0.0, 0.0, 0.0, 0.02545573613106984, 0.30808845160815984, 1.174969336492933, 2.0061084741262825, 2.94330578929200

/tmp/ipykernel_3752326/1862305445.py:6: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  monthly_sum = system_power_df.resample('M').sum()


{'monthly_avgs': {1: [0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.053701844989045865,
   0.3635036652453755,
   0.666060962530182,
   0.7992231402134995,
   0.8522076905613841,
   0.7318497387468278,
   0.6609871458543845,
   0.37951716231918564,
   0.004917046524599206,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0],
  2: [0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.013645871189346284,
   0.2951942286716836,
   0.6490290065641929,
   0.7872508409343323,
   0.9424003558601068,
   0.9794867862039264,
   0.9151097398762081,
   0.8416199761466007,
   0.6426685125715086,
   0.3882742860724059,
   0.00851171583418764,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0],
  3: [0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.018552094120648496,
   0.32044678341577953,
   1.001533404313787,
   1.7428788309675591,
   2.2345758164988014,
   2.6085326597428544,
   2.6680151402762515,
   2.6137557854787024,
   2.262404245164015,
   1.864541899087937,
   1.2482474